In [1]:
import pandas as pd
import re

#get dataframes for UNITE taxonomy and FungalTraits
unite_taxonomy = pd.read_csv('/Users/svenst/Documents/qiime2-wrkdir/global-gutfungi/UNITE-FungalTraits/aa683f84-5373-4e3f-a43d-bcc28bb9a658/data/taxonomy.tsv', delimiter='\t')  # Replace 'path_to_your_file.csv' with your file path

fungal_traits = pd.read_excel('/Users/svenst/Documents/qiime2-wrkdir/global-gutfungi/UNITE-FungalTraits/FungalTraits-classification.xlsx')  # Replace 'path_to_your_file.xlsx' with your file path

#define function to extract genus from taxonomic label
def extract_genus(taxon):
    match = re.search(r'(g__[^;]+)', taxon)
    if match:
        return match.group(1)  # This keeps 'g__' with the genus name
    return None  # Return None if 'g__' is not found

#extract genus from UNITE taxonomy
unite_taxonomy['Genus'] = unite_taxonomy['Taxon'].apply(extract_genus)

#see if it worked
print(unite_taxonomy.head())

# Get the index of the 'GENUS' column
index_of_genus = fungal_traits.columns.get_loc("GENUS") + 1

# Insert 'g__Genus' right after 'GENUS'
fungal_traits.insert(index_of_genus, 'g__Genus', 'g__' + fungal_traits['GENUS'])


fungal_traits_filled = fungal_traits.fillna('not reported')


# Merging 'unite_taxonomy' and 'fungal_traits_filled' based on common genus column
merged_df = pd.merge(unite_taxonomy, fungal_traits_filled, left_on='Genus', right_on='g__Genus', how='left')

#look at file
print(merged_df.head())

#save
merged_df.to_csv('unite_fungaltraits_merged.tsv', sep='\t', index=False)